# Data Challenge

## Scraper Marketplace

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import os
import requests
import logging
import cv2
import numpy as np
import tensorflow as tf
import shutil
# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Chemin vers le ChromeDriver
driver_path = 'C:/Webdrivers/chromedriver.exe'  # Assurez-vous que ce chemin est correct

# Configuration de Selenium
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
# options.add_argument('--headless')  # Exécute Chrome en mode headless (optionnel)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--disable-features=NetworkService')
options.add_argument('--window-size=1920x1080')

# Spécifier le chemin du driver via les options
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# URL de connexion à Facebook
login_url = 'https://www.facebook.com/login'

# Vos informations de connexion
username = '48647376'
password = 'Ma@22117035'

try:
    # Accéder à la page de connexion de Facebook
    driver.get(login_url)
    time.sleep(3)

    # Remplir le formulaire de connexion
    email_input = driver.find_element(By.ID, 'email')
    password_input = driver.find_element(By.ID, 'pass')
    login_button = driver.find_element(By.NAME, 'login')

    email_input.send_keys(username)
    password_input.send_keys(password)
    login_button.click()

    time.sleep(5)  # Attendre le chargement complet de la page

    # Accéder à la page de Facebook Marketplace
    marketplace_url = 'https://www.facebook.com/marketplace/category/vehicles'
    driver.get(marketplace_url)
    time.sleep(5)

    # Faire défiler la page pour charger toutes les publications
    scroll_pause_time = 2
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Créer un dossier pour les images téléchargées
    folder_path = 'car_images'
    os.makedirs(folder_path, exist_ok=True)

    # Trouver les liens des annonces de véhicules
    post_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/marketplace/item/")]')
    post_links = [post.get_attribute('href') for post in post_links]  # Copie des liens des annonces

    img_counter = 0
    max_images = 100000 # Limite du nombre d'images à télécharger
    session = requests.Session()  # Utilisation de sessions persistantes pour optimiser le téléchargement

    # Télécharger les images de chaque annonce
    for index, post_url in enumerate(post_links):
        if img_counter >= max_images:
            logging.info(f"Limite de {max_images} images atteinte.")
            break
        
        logging.info(f"Accès à l'annonce {index + 1}/{len(post_links)} : {post_url}")
        
        try:
            driver.get(post_url)
            time.sleep(3)

            # Vérifiez le code HTML de la page de l'annonce pour les images
            images = driver.find_elements(By.XPATH, '//img[contains(@src, "scontent")]')
            logging.info(f"Trouvé {len(images)} images dans l'annonce {index + 1}")

            for img in images:
                src = img.get_attribute('src')
                if src and src.startswith('http'):
                    # Télécharger l'image
                    img_data = session.get(src).content
                    file_name = f'web1_image_{img_counter}.jpeg'
                    img_path = os.path.join(folder_path, file_name)
                    with open(img_path, 'wb') as handler:
                        handler.write(img_data)
                    img_counter += 1

                    if img_counter >= max_images:
                        break

            driver.back()
            time.sleep(2)
        
        except (NoSuchElementException, StaleElementReferenceException) as e:
            logging.error(f"Erreur lors de l'accès à l'annonce {index + 1}: {e}")

    logging.info(f'Téléchargement de {img_counter} images terminé.')

finally:
    # Fermer le navigateur
    driver.quit()

2024-05-27 22:03:48,884 - INFO - Accès à l'annonce 1/397 : https://web.facebook.com/marketplace/item/3905557593006626/?ref=category_feed&referral_code=undefined&referral_story_type=listing&tracking=%7B%22qid%22%3A%22-1922980274996425417%22%2C%22mf_story_key%22%3A%227488785437842653%22%2C%22commerce_rank_obj%22%3A%22%7B%5C%22target_id%5C%22%3A7488785437842653%2C%5C%22target_type%5C%22%3A0%2C%5C%22primary_position%5C%22%3A0%2C%5C%22ranking_signature%5C%22%3A2526174802599716764%2C%5C%22commerce_channel%5C%22%3A504%2C%5C%22value%5C%22%3A0.00019832159797477%2C%5C%22candidate_retrieval_source_map%5C%22%3A%7B%5C%227488785437842653%5C%22%3A111%7D%7D%22%7D&__tn__=!%3AD
2024-05-27 22:03:57,115 - INFO - Trouvé 33 images dans l'annonce 1
2024-05-27 22:04:06,078 - INFO - Accès à l'annonce 2/397 : https://web.facebook.com/marketplace/item/732353485708311/?ref=category_feed&referral_code=undefined&referral_story_type=listing&tracking=%7B%22qid%22%3A%22-1922980274996425417%22%2C%22mf_story_key%22%3A%2

## Nettoyage des données

In [ ]:
# Chemin vers le dossier des images existantes
source_folder = 'car_images1'
destination_folder = 'Web-scraped images'
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Charger le modèle MobileNetV2 pré-entraîné pour la classification d'images
model = tf.keras.applications.MobileNetV2(weights="imagenet")

# Fonction pour prédire si une image contient une voiture
def is_car_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    preds = model.predict(img)
    decoded_preds = tf.keras.applications.mobilenet_v2.decode_predictions(preds, top=1)
    return decoded_preds[0][0][1] in ["car", "vehicle", "sports_car", "jeep", "cab", "convertible"]

# Fonction pour nettoyer les images
def clean_images():
    img_counter = 0
    for image_file in os.listdir(source_folder):
        image_path = os.path.join(source_folder, image_file)
        if is_car_image(image_path):
            shutil.copy(image_path, os.path.join(destination_folder, image_file))
            img_counter += 1
            print(f"Image {image_file} transférée vers le dossier nettoyé.")
        else:
            os.remove(image_path)
            print(f"Image {image_file} supprimée.")

    print(f"Nettoyage terminé. {img_counter} images ont été conservées.")

# Nettoyer les images
clean_images()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image web1_image_0.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Image web1_image_1.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Image web1_image_10.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Image web1_image_100.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Image web1_image_1000.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Image web1_image_1001.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Image web1_image_1002.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Image web1_image_1003.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Image web1_image_1004.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Image web1_image_1005.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Image web1_image_1006.jpeg supprimée.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Image web1_image_1007.jpeg transférée vers le dossier nettoyé.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Image web1_image_1008.jpeg supprimée

In [2]:
# Chemin vers le dossier contenant les images
folder_path = 'Web-scraped images'  # Remplacez par le chemin de votre dossier

# Obtenir la liste des fichiers dans le dossier
files = os.listdir(folder_path)

# Filtrer les fichiers d'image uniquement
image_files = [f for f in files if f.lower().endswith(('.jpeg', '.jpg', '.png', '.gif'))]

# Trier les fichiers par nom (optionnel, selon votre critère de tri)
image_files.sort()

# Renommer les fichiers
for idx, file in enumerate(image_files):
    file_extension = os.path.splitext(file)[1]  # Obtenir l'extension du fichier
    new_name = f'web1_{idx + 1}{file_extension}'  # Nouveau nom de fichier
    src = os.path.join(folder_path, file)
    dst = os.path.join(folder_path, new_name)
    os.rename(src, dst)
    print(f"Renommé {file} en {new_name}")

print("Renommage terminé.")

Renommé web1_image_1.jpg en web1_1.jpg
Renommé web1_image_10.jpg en web1_2.jpg
Renommé web1_image_100.jpg en web1_3.jpg
Renommé web1_image_1000.jpg en web1_4.jpg
Renommé web1_image_1001.jpg en web1_5.jpg
Renommé web1_image_1002.jpg en web1_6.jpg
Renommé web1_image_1003.jpg en web1_7.jpg
Renommé web1_image_1004.jpg en web1_8.jpg
Renommé web1_image_1005.jpg en web1_9.jpg
Renommé web1_image_1006.jpg en web1_10.jpg
Renommé web1_image_1007.jpg en web1_11.jpg
Renommé web1_image_1008.jpg en web1_12.jpg
Renommé web1_image_1009.jpg en web1_13.jpg
Renommé web1_image_101.jpg en web1_14.jpg
Renommé web1_image_1010.png en web1_15.png
Renommé web1_image_1011.jpg en web1_16.jpg
Renommé web1_image_1012.jpg en web1_17.jpg
Renommé web1_image_1013.jpg en web1_18.jpg
Renommé web1_image_1014.jpg en web1_19.jpg
Renommé web1_image_1015.jpg en web1_20.jpg
Renommé web1_image_1016.jpg en web1_21.jpg
Renommé web1_image_1017.jpg en web1_22.jpg
Renommé web1_image_1018.png en web1_23.png
Renommé web1_image_1019.png

In [3]:
# Chemin vers le dossier contenant les images
folder_path = 'Photos taken by your camera'  # Remplacez par le chemin de votre dossier

# Obtenir la liste des fichiers dans le dossier
files = os.listdir(folder_path)

# Filtrer les fichiers d'image uniquement
image_files = [f for f in files if f.lower().endswith(('.jpeg', '.jpg', '.png', '.gif'))]

# Trier les fichiers par nom (optionnel, selon votre critère de tri)
image_files.sort()

# Renommer les fichiers
for idx, file in enumerate(image_files):
    file_extension = os.path.splitext(file)[1]  # Obtenir l'extension du fichier
    new_name = f'cam1_{idx + 1}{file_extension}'  # Nouveau nom de fichier
    src = os.path.join(folder_path, file)
    dst = os.path.join(folder_path, new_name)
    os.rename(src, dst)
    print(f"Renommé {file} en {new_name}")

print("Renommage terminé.")

Renommé 0001059f437adecbe8cb75d4285b1f10f9e23353.jpg en cam1_1.jpg
Renommé 0003c35a2b8bc9302c1dfaf92bc2b44c0227e16a.jpg en cam1_2.jpg
Renommé 0003c35a2b8bc9302c1dfaf92bc2b44c0227e16a.png en cam1_3.png
Renommé 00238ac361d42cab40ad91d3384f3b85e06b7c83.jpg en cam1_4.jpg
Renommé 0039b9476510afbcda0ead3ffceb4c0faa0c9686.jpg en cam1_5.jpg
Renommé 004cb91c234cab5ccf973b034251401fd88e8069.jpg en cam1_6.jpg
Renommé 004e267592158d87cf6eb89976b5f9efd0ddb5ca.jpg en cam1_7.jpg
Renommé 005706548d51dfe3095833ba4f8d7e6032fed29e.jpg en cam1_8.jpg
Renommé 008fa0e44b41b3365cbe9bff7f79ae2bb859aa22.jpg en cam1_9.jpg
Renommé 009a08f2c4c99718f2c29c364153eddea10875ca.jpg en cam1_10.jpg
Renommé 00b0d5346384f51a5fea89d6582bc51c4545bd7a.jpg en cam1_11.jpg
Renommé 00b0d5346384f51a5fea89d6582bc51c4545bd7a.png en cam1_12.png
Renommé 00c51f8895908098d3e9d20b7d2065726563e604.jpg en cam1_13.jpg
Renommé 00d6290617807642cb9b71ec16c75edfbe09c4a9.jpg en cam1_14.jpg
Renommé 00d8948b8ba5141347130432ca8abc1eebfe43fb.jpg en c

In [2]:
import os

def split_zip_file(input_zip_path, output_dir, chunk_size):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Lire le fichier zip d'origine
    with open(input_zip_path, 'rb') as f:
        zip_data = f.read()
    
    # Déterminer le nombre de morceaux
    total_size = len(zip_data)
    num_chunks = total_size // chunk_size + (1 if total_size % chunk_size else 0)
    
    for i in range(num_chunks):
        chunk_data = zip_data[i*chunk_size : (i+1)*chunk_size]
        chunk_file_path = os.path.join(output_dir, f'split_{i+1}.zip')
        
        with open(chunk_file_path, 'wb') as chunk_file:
            chunk_file.write(chunk_data)
        
        print(f"Chunk {i+1} created: {chunk_file_path}")

# Spécifiez le chemin vers votre fichier zip d'origine et le répertoire de sortie
input_zip_path = 'C:/Users/hp/Downloads/Data Challenge/Data_Challenge.zip'  # Remplacez par le chemin correct de votre fichier zip
output_dir = 'C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips'  # Remplacez par le chemin correct du répertoire de sortie
chunk_size = 1 * 1024 * 1024 * 1024  # 1 Go

# Diviser le fichier zip
split_zip_file(input_zip_path, output_dir, chunk_size)

Chunk 1 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\split_1.zip
Chunk 2 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\split_2.zip
Chunk 3 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\split_3.zip
Chunk 4 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\split_4.zip
Chunk 5 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\split_5.zip


In [1]:
import os
import zipfile

def create_zip_chunk(files, chunk_index, output_dir):
    chunk_file_path = os.path.join(output_dir, f'Submission_{chunk_index}.zip')
    with zipfile.ZipFile(chunk_file_path, 'w', zipfile.ZIP_DEFLATED) as chunk_zip:
        for file_info, file_data in files:
            chunk_zip.writestr(file_info, file_data)
    print(f"Chunk {chunk_index} created: {chunk_file_path}")

def split_zip_file(input_zip_path, output_dir, max_chunk_size):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with zipfile.ZipFile(input_zip_path, 'r') as zip_ref:
        chunk_files = []
        chunk_size = 0
        chunk_index = 1
        
        for file_info in zip_ref.infolist():
            file_data = zip_ref.read(file_info.filename)
            file_size = len(file_data)
            
            if chunk_size + file_size > max_chunk_size:
                create_zip_chunk(chunk_files, chunk_index, output_dir)
                chunk_files = []
                chunk_size = 0
                chunk_index += 1

            chunk_files.append((file_info, file_data))
            chunk_size += file_size
        
        if chunk_files:
            create_zip_chunk(chunk_files, chunk_index, output_dir)

# Spécifiez le chemin vers votre fichier zip d'origine et le répertoire de sortie
input_zip_path = 'C:/Users/hp/Downloads/Data Challenge/Data_Challenge.zip'  # Remplacez par le chemin correct de votre fichier zip
output_dir = 'C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips'  # Remplacez par le chemin correct du répertoire de sortie
max_chunk_size = 0.8 * 1024 * 1024 * 1024  # 0.7 Go

# Diviser le fichier zip
split_zip_file(input_zip_path, output_dir, max_chunk_size)

Chunk 1 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\Submission_1.zip
Chunk 2 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\Submission_2.zip
Chunk 3 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\Submission_3.zip
Chunk 4 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\Submission_4.zip
Chunk 5 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\Submission_5.zip
Chunk 6 created: C:/Users/hp/Downloads/Data Challenge/Submissions/split_zips\Submission_6.zip
